<a href="https://colab.research.google.com/github/JuliaoAmaral/doutorado/blob/main/coleta_metadados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O código abaixo usa Expressões Regulares para buscar os metadados dos artigos através de caracateres especiais inseridos no arquivo de word no momento do tratamento dos dados.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-docx

import os
import re
import pandas as pd
from docx import Document

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.2 MB/s eta 0:00:00


In [3]:
# Função para extrair metadados de um arquivo .docx
def extrair_metadados(caminho_arquivo):
    doc = Document(caminho_arquivo)
    texto_docx = ""
    for paragrafo in doc.paragraphs:
        texto_docx += paragrafo.text + "\n"

    padrao_periodico = r'<rev>\s*([^<]+)\s*<\\rev>'
    match_periodico = re.search(padrao_periodico, texto_docx)
    nome_periodico = match_periodico.group(1).strip() if match_periodico else None

    padrao_titulo = r'<tit>(.*?)<\\tit>'
    titulos_encontrados = re.findall(padrao_titulo, texto_docx, re.DOTALL)

    padrao_autores = r'<aut>(.*?)<\\aut>'
    autores_encontrados = re.findall(padrao_autores, texto_docx, re.DOTALL)

    padrao_orcid = r'<orc>\s*ORCID:\s*(.*?)\s*<\\orc>'
    match_orcid = re.search(padrao_orcid, texto_docx)
    numero_orcid = match_orcid.group(1).strip() if match_orcid else None

    padrao_afiliacao = r'<afi>(.*?)<\\afi>'
    afiliacoes_encontradas = re.findall(padrao_afiliacao, texto_docx, re.DOTALL)

    padrao_resumo = r'<abs>(.*?)<\\abs>'
    resumo_encontrado = re.search(padrao_resumo, texto_docx, re.DOTALL)
    resumo = resumo_encontrado.group(1) if resumo_encontrado else None

    padrao_palavras_chave = r'<key>(.*?)<\\key>'
    palavras_chave_encontradas = re.findall(padrao_palavras_chave, texto_docx, re.DOTALL)

    padrao_volume = r'<vol>(.*?)<\\vol>'
    match_volume = re.search(padrao_volume, texto_docx)
    volume = match_volume.group(1).strip() if match_volume else None

    padrao_numero = r'<num>\s*(\d+)\s*<\\num>'
    match_numero = re.search(padrao_numero, texto_docx)
    numero = match_numero.group(1) if match_numero else None

    padrao_ano = r'<ano>\s*Ano:\s*(\d{4})\s*<\\ano>'
    match_ano = re.search(padrao_ano, texto_docx)
    ano_publicacao = match_ano.group(1) if match_ano else None

    padrao_doi = r'<doi>\s*DOI:\s*([^\s<]+)\s*<\\doi>'
    match_doi = re.search(padrao_doi, texto_docx)
    doi = match_doi.group(1) if match_doi else None

    padrao_referencia = r'<i>(.*?)<\\f>'
    referencias_encontradas = re.findall(padrao_referencia, texto_docx, re.DOTALL)

    return {
        'Nome do Periódico': nome_periodico,
        'Títulos': titulos_encontrados,
        'Autores': autores_encontrados,
        'ORCID': numero_orcid,
        'Afiliações': afiliacoes_encontradas,
        'Resumo': resumo,
        'Palavras-Chave': palavras_chave_encontradas,
        'Volume': volume,
        'Número': numero,
        'Ano de Publicação': ano_publicacao,
        'DOI': doi,
        'Referências': referencias_encontradas
    }

In [4]:
# Pasta contendo os arquivos .docx no Google Drive
pasta_arquivos = '/content/drive/MyDrive/Doutorado/Análise/Artigos para Análise'

In [5]:
# Lista para armazenar os metadados de todos os arquivos
lista_metadados = []

# Iterar sobre os arquivos na pasta
for arquivo in os.listdir(pasta_arquivos):
    if arquivo.endswith(".docx"):
        caminho_arquivo = os.path.join(pasta_arquivos, arquivo)
        metadados = extrair_metadados(caminho_arquivo)
        lista_metadados.append(metadados)

# Converter a lista de metadados em DataFrame
df_metadados = pd.DataFrame(lista_metadados)

# Nome do arquivo Excel de saída
nome_arquivo_excel = '/content/drive/MyDrive/Doutorado/Análise/Metadados_Artigos.xlsx'

# Exportar os metadados para o arquivo Excel
df_metadados.to_excel(nome_arquivo_excel, index=False)

print("Exportação concluída com sucesso!")

Exportação concluída com sucesso!
